In [1]:
import csv
import pandas as pd
import numpy as np
from collections import defaultdict

In [16]:
df_CME = pd.read_csv("Input Data/ap2_track.csv", header = None)
M_CME = df_CME.to_numpy()
M_CME[:, 2] += M_CME[:, 1]-1
M_CME[:, 5] = 169 - M_CME[:, 5]

df_Dino = pd.read_csv("Input Data/ap2_intensity_fix.csv")
df_Dino = df_Dino.drop("max_intensity", axis = 1)
M_Dino = df_Dino.to_numpy()
M_Dino[:, 1:3] += 1

comp_list = np.zeros((len(M_CME),19), dtype=object)

In [25]:
t_vec = defaultdict(list)
threshold = 5
multi_match_list = []
for vec_Dino in M_Dino:
    t_vec[vec_Dino[2]].append(vec_Dino)

for t_val in t_vec:
    t_vec[t_val] = np.array(t_vec[t_val])

for i, vec_CME in enumerate(M_CME):
    t_val = vec_CME[2]
    dino_group = t_vec[t_val]
    diffs = dino_group[:, 3:6] - vec_CME[3:6]
    dists = np.linalg.norm(diffs, axis=1)
    min_idx = np.argmin(dists)
    min_dist = dists[min_idx]
    best_vec_Dino = dino_group[min_idx]
    comp_list[i, 0:8] = vec_CME
    comp_list[i, 8:16] = best_vec_Dino
    comp_list[i, 16] = min_dist

    # New list by threshold
    below = np.where(dists < threshold)[0]
    if len(below) > 0:
        dino_ids = ",".join(str(int(x)) for x in dino_group[below, 0])
        dists_str = ",".join(f"{dists[j]:.2f}" for j in below)
    else:
        dino_ids = ""
        dists_str = ""
    comp_list[i, 17] = dino_ids
    comp_list[i, 18] = dists_str

multi_match_array = np.array(multi_match_list, dtype=object)

In [4]:
df_comp = pd.DataFrame(comp_list, columns = ["ID (CME)","t_start (CME)", "t", "x (CME)", "y (CME)", 
                                             "z (CME)", "FI (CME)", "Track Length (CME)", "ID (Dino)", 
                                             "t_start (Dino)", "t_ig", "x (Dino)", "y (Dino)", "z (Dino)", 
                                             "FI (Dino)", "Track Length (Dino)", "Distance", "Multi ID (Dino)", "Multi Distance (Dino)"])

df_comp = df_comp.drop("t_ig", axis=1)
df_comp = df_comp[["ID (CME)","ID (Dino)", "x (CME)", "y (CME)", 
                   "z (CME)", "x (Dino)", "y (Dino)", "z (Dino)", 
                   "t", "t_start (CME)", "t_start (Dino)","FI (CME)", 
                   "FI (Dino)","Track Length (CME)", "Track Length (Dino)", "Distance","Multi ID (Dino)", "Multi Distance (Dino)"]] 
df_sorted = df_comp.sort_values(by=["ID (CME)","t"], ascending=[True, True])

In [5]:
df_sorted.to_csv("Output Data/Comparison.csv", index=False)

In [6]:
df = df_sorted[df_sorted["ID (CME)"]==7]
df

,ID (CME),ID (Dino),x (CME),y (CME),z (CME),x (Dino),y (Dino),z (Dino),t,t_start (CME),t_start (Dino),FI (CME),FI (Dino),Track Length (CME),Track Length (Dino),Distance,Multi ID (Dino),Multi Distance (Dino)
594,7.0,464.0,344.66,353.28,55.51,343.69022,352.39755,55.49809,1.0,1.0,1.0,313.27,411.5,99.0,99.0,1.311233,464,1.31
595,7.0,464.0,345.31,352.67,55.56,344.4965,351.71472,55.94096,2.0,1.0,1.0,282.1,400.5,99.0,99.0,1.311287,464,1.31
596,7.0,464.0,345.48,352.54,54.9,344.72147,351.9407,55.780052,3.0,1.0,1.0,227.04,392.2,99.0,99.0,1.307295,464,1.31
597,7.0,464.0,345.16,352.52,55.65,344.0845,351.68625,56.04506,4.0,1.0,1.0,276.59,416.0,99.0,99.0,1.417008,464,1.42
598,7.0,464.0,345.41,352.43,55.66,344.4479,351.65475,55.89337,5.0,1.0,1.0,262.03,392.8,99.0,99.0,1.257422,464,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,7.0,464.0,345.03,354.65,54.57,343.60358,353.85574,54.85887,95.0,1.0,1.0,100.75,234.9,99.0,99.0,1.658001,464,1.66
689,7.0,464.0,346.17,353.74,53.51,344.59393,352.97256,55.171696,96.0,1.0,1.0,78.646,231.0,99.0,99.0,2.415408,464,2.42
690,7.0,464.0,345.17,353.63,53.3,343.7944,352.77805,54.669,97.0,1.0,1.0,76.552,227.9,99.0,99.0,2.119494,464,2.12
691,7.0,464.0,345.35,352.74,53.39,344.15112,352.34158,55.14416,98.0,1.0,1.0,75.39,216.9,99.0,99.0,2.161742,464,2.16
